<a href="https://colab.research.google.com/github/AUT-Student/NN-HW5/blob/main/NN_HW5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix

import tensorflow as tf
import tensorflow.keras as keras
from keras.layers import Dense, Input, RNN
import keras.backend as K
from keras.models import Sequential
from keras.callbacks import EarlyStopping

# Dataset

In [2]:
!gdown https://archive.ics.uci.edu/ml/machine-learning-databases/robotfailure-mld/lp1.data
!gdown https://archive.ics.uci.edu/ml/machine-learning-databases/robotfailure-mld/lp2.data
!gdown https://archive.ics.uci.edu/ml/machine-learning-databases/robotfailure-mld/lp3.data
!gdown https://archive.ics.uci.edu/ml/machine-learning-databases/robotfailure-mld/lp4.data
!gdown https://archive.ics.uci.edu/ml/machine-learning-databases/robotfailure-mld/lp5.data

Downloading...
From: https://archive.ics.uci.edu/ml/machine-learning-databases/robotfailure-mld/lp1.data
To: /content/lp1.data
100% 27.3k/27.3k [00:00<00:00, 488kB/s]
Downloading...
From: https://archive.ics.uci.edu/ml/machine-learning-databases/robotfailure-mld/lp2.data
To: /content/lp2.data
100% 14.6k/14.6k [00:00<00:00, 257kB/s]
Downloading...
From: https://archive.ics.uci.edu/ml/machine-learning-databases/robotfailure-mld/lp3.data
To: /content/lp3.data
100% 14.5k/14.5k [00:00<00:00, 267kB/s]
Downloading...
From: https://archive.ics.uci.edu/ml/machine-learning-databases/robotfailure-mld/lp4.data
To: /content/lp4.data
100% 33.6k/33.6k [00:00<00:00, 570kB/s]
Downloading...
From: https://archive.ics.uci.edu/ml/machine-learning-databases/robotfailure-mld/lp5.data
To: /content/lp5.data
100% 49.2k/49.2k [00:00<00:00, 394kB/s]


In [3]:
normal_label_datasets = ["normal","normal","ok","normal","normal"]

dataset_x = []
dataset_y = []
for i in range(5):
  normal_label = normal_label_datasets[i]
  dataset_i = pd.read_csv(f"/content/lp{i+1}.data", header=None)
  
  new_label = None
  new_data = None

  for j, row in dataset_i.iterrows():
    if j%16==0:
      new_data = []
      
      if row[0] == normal_label:
         new_label = 0
      else:
         new_label = 1

    else:
      new_data.append([float(item) for item in row[0].split()])

      if j%16==15:
        dataset_x.append(new_data)
        dataset_y.append(new_label)

In [4]:
dataset_x, dataset_y = shuffle(dataset_x, dataset_y, random_state=0)

In [5]:
dataset_len = len(dataset_x)

In [6]:
train_x = dataset_x[:int(0.7*dataset_len)]
train_y = dataset_y[:int(0.7*dataset_len)]

valid_x = dataset_x[int(0.7*dataset_len):int(0.8*dataset_len)]
valid_y = dataset_y[int(0.7*dataset_len):int(0.8*dataset_len)]

test_x = dataset_x[int(0.8*dataset_len):]
test_y = dataset_y[int(0.8*dataset_len):]

In [7]:
sum(train_y) * 100 / len(train_y)

71.60493827160494

In [8]:
sum(test_y) * 100 / len(test_y)

77.41935483870968

# Elman

In [9]:
class ElmanCell(keras.layers.Layer):
  # Refrences: https://github.com/keras-team/keras/blob/v2.9.0/keras/layers/rnn/base_rnn.py#L36-L862
  # Refrences: https://github.com/keras-team/keras/blob/v2.9.0/keras/layers/rnn/simple_rnn.py#L242-L492

  def __init__(self, units):
      super().__init__()
      self.units = units
      self.state_size = units

  def build(self, input_shape):
      self.kernel = self.add_weight(shape=(input_shape[-1], self.units), initializer="glorot_uniform", name="kernel")
      self.recurrent_kernel = self.add_weight(shape=(self.units, self.units), initializer="orthogonal", name="recurrent_kernel")
      self.bias = self.add_weight(shape=(self.units), initializer="zeros", name="bias")
      self.built = True

  def call(self, inputs, states):
      previous_hidden = states[0]
      current_hidden = K.dot(inputs, self.kernel) + K.dot(previous_hidden, self.recurrent_kernel)
      current_hidden = K.bias_add(current_hidden, self.bias)

      current_hidden = K.sigmoid(current_hidden)
      return current_hidden, [current_hidden]

In [10]:
class ElmanNetwork(keras.Model):
  def __init__(self, cell_number):
    super().__init__()
    self.model = Sequential([
                             Input((15,6,)),
                             RNN(ElmanCell(cell_number)),
                             Dense(2, activation="softmax")
                            ])
  def call(self, inputs):
    return self.model.call(inputs)

In [27]:
elman = ElmanNetwork(64)

In [28]:
elman.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [29]:
elman.fit(x=train_x, y=train_y, epochs=100, batch_size=16, validation_data=(valid_x, valid_y))

Epoch 1/100
21/21 [==============================] - 3s 24ms/step - loss: 0.5801 - accuracy: 0.6574 - val_loss: 0.5586 - val_accuracy: 0.6304
Epoch 2/100
21/21 [==============================] - 0s 9ms/step - loss: 0.4517 - accuracy: 0.7346 - val_loss: 0.5308 - val_accuracy: 0.5435
Epoch 3/100
21/21 [==============================] - 0s 9ms/step - loss: 0.4174 - accuracy: 0.7778 - val_loss: 0.5127 - val_accuracy: 0.5652
Epoch 4/100
21/21 [==============================] - 0s 9ms/step - loss: 0.3940 - accuracy: 0.8086 - val_loss: 0.4926 - val_accuracy: 0.6522
Epoch 5/100
21/21 [==============================] - 0s 7ms/step - loss: 0.3751 - accuracy: 0.8117 - val_loss: 0.4884 - val_accuracy: 0.6957
Epoch 6/100
21/21 [==============================] - 0s 8ms/step - loss: 0.3576 - accuracy: 0.8272 - val_loss: 0.4740 - val_accuracy: 0.6957
Epoch 7/100
21/21 [==============================] - 0s 8ms/step - loss: 0.3467 - accuracy: 0.8364 - val_loss: 0.4655 - val_accuracy: 0.7174
Epoch 8/100


In [30]:
elman.evaluate(x=train_x, y=train_y)
elman.evaluate(x=valid_x, y=valid_y)
elman.evaluate(x=test_x, y=test_y)

3/3 [==============================] - 0s 6ms/step - loss: 0.3403 - accuracy: 0.9355


[0.34029626846313477, 0.9354838728904724]

# Jordan

In [41]:
class JordanCell(keras.layers.Layer):
  # Refrences: https://github.com/keras-team/keras/blob/v2.9.0/keras/layers/rnn/base_rnn.py#L36-L862

  def __init__(self, units):
      super().__init__()
      self.units = units
      self.state_size = 2

  def build(self, input_shape):
      self.kernel = self.add_weight(shape=(input_shape[-1], self.units),name="kernel")
      self.recurrent_kernel = self.add_weight(shape=(2, self.units), name="recurrent_kernel")
      self.output_kernel = self.add_weight(shape=(self.units, 2), name="output_kernel")
      self.built = True

  def call(self, inputs, states):
      prev_output = states[0]
      h = K.dot(inputs, self.kernel)
      output = h + K.dot(prev_output, self.recurrent_kernel)
      
      output = K.dot(output, self.output_kernel)
      output = K.softmax(output)

      return output, [output]

In [42]:
class JordanNetwork(keras.Model):
  def __init__(self, cell_number):
    super().__init__()
    self.model = Sequential([
                             Input((15,6,)),
                             RNN(JordanCell(cell_number)),
                             Dense(2, activation="softmax")
                            ])
  def call(self, inputs):
    return self.model.call(inputs)

In [43]:
jordan = JordanNetwork(64)

In [45]:
jordan.compile(optimizer=keras.optimizers.Adam(learning_rate=0.00001), loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [46]:
jordan.fit(x=train_x, y=train_y, epochs=100, batch_size=16, validation_data=(valid_x, valid_y))

Epoch 1/100
21/21 [==============================] - 2s 19ms/step - loss: 0.6942 - accuracy: 0.5957 - val_loss: 0.7247 - val_accuracy: 0.5217
Epoch 2/100
21/21 [==============================] - 0s 6ms/step - loss: 0.6937 - accuracy: 0.5957 - val_loss: 0.7240 - val_accuracy: 0.5217
Epoch 3/100
21/21 [==============================] - 0s 6ms/step - loss: 0.6931 - accuracy: 0.5957 - val_loss: 0.7234 - val_accuracy: 0.5217
Epoch 4/100
21/21 [==============================] - 0s 6ms/step - loss: 0.6925 - accuracy: 0.5957 - val_loss: 0.7227 - val_accuracy: 0.5217
Epoch 5/100
21/21 [==============================] - 0s 7ms/step - loss: 0.6919 - accuracy: 0.5957 - val_loss: 0.7221 - val_accuracy: 0.5217
Epoch 6/100
21/21 [==============================] - 0s 6ms/step - loss: 0.6912 - accuracy: 0.5957 - val_loss: 0.7214 - val_accuracy: 0.5217
Epoch 7/100
21/21 [==============================] - 0s 6ms/step - loss: 0.6904 - accuracy: 0.5957 - val_loss: 0.7207 - val_accuracy: 0.5217
Epoch 8/100


## Jordan Network

In [17]:
class Jordan(keras.Model): 
  def __init__(self):
    super().__init__(self)

    self.model = Sequential([
                             Input(8, name="Input"),
                             Dense(64, name="Hidden1"),
                             Dense(64, name="Hidden2"),
                             Dense(32, name="Hidden3"),
                             Dense(2, activation="softmax", name="Output")
    ])

  def call_recercive(self, input, prev_input):
    if len(prev_input)==0:
      concated = K.concatenate([input, K.constant([0., 0.])])
      concated = tf.expand_dims(concated, axis=0)
    else:
      prev_output = self.call_recercive(prev_input[-1], prev_input[:-1])

      input = tf.expand_dims(input, axis=0)
      concated = K.concatenate([input, prev_output])
    
    output = self.model.call(concated)  
    return output

  def call(self, inputs):
    inputs = inputs[0]

    input = inputs[-1]
    prev_input = inputs[:-1]

    return self.call_recercive(input, prev_input)

In [18]:
jordan = Jordan()

In [19]:
jordan.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0003), loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [20]:
es_callback = EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)

In [21]:
log_dir = "./log"
tb_callback = keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
!rm -r ./log

In [22]:
jordan.fit(x=train_x, y=train_y, epochs=50, batch_size=1, validation_data=(valid_x, valid_y), callbacks=[es_callback, tb_callback])

Epoch 1/50
324/324 [==============================] - 4s 5ms/step - loss: 3.4093 - accuracy: 0.6080 - val_loss: 1.6314 - val_accuracy: 0.4348
Epoch 2/50
324/324 [==============================] - 1s 3ms/step - loss: 1.1526 - accuracy: 0.6944 - val_loss: 1.3833 - val_accuracy: 0.5870
Epoch 3/50
324/324 [==============================] - 1s 3ms/step - loss: 3.9900 - accuracy: 0.6914 - val_loss: 1.8111 - val_accuracy: 0.6087
Epoch 4/50
324/324 [==============================] - 1s 3ms/step - loss: 1.0320 - accuracy: 0.7160 - val_loss: 3.5502 - val_accuracy: 0.4565
Epoch 5/50
324/324 [==============================] - 1s 3ms/step - loss: 0.9003 - accuracy: 0.7315 - val_loss: 1.2702 - val_accuracy: 0.4783
Epoch 6/50
324/324 [==============================] - 1s 3ms/step - loss: 1.4149 - accuracy: 0.7284 - val_loss: 1.4060 - val_accuracy: 0.4783
Epoch 7/50
324/324 [==============================] - 1s 3ms/step - loss: 1.6680 - accuracy: 0.6975 - val_loss: 1.1196 - val_accuracy: 0.5652
Epoch 

In [28]:
%tensorboard --logdir ./log

UsageError: Line magic function `%tensorboard` not found.


In [25]:
jordan.evaluate(x=train_x, y=train_y, batch_size=1)
jordan.evaluate(x=valid_x, y=valid_y, batch_size=1)
jordan.evaluate(x=test_x, y=test_y, batch_size=1)

93/93 [==============================] - 0s 2ms/step - loss: 0.4885 - accuracy: 0.7204


[0.4884602427482605, 0.7204301357269287]

In [26]:
test_predict = [np.argmax(item) for item in jordan.predict(test_x, batch_size=1)]

In [27]:
confusion_matrix(test_y, test_predict)

array([[ 4, 17],
       [ 9, 63]])